# generate_error_table

## imports

In [1]:
import pandas as pd
import numpy as np

## load data

In [8]:
df0 = pd.read_csv('data/benchmarks_20230122.csv')
df1 = pd.read_csv('data/benchmarks_20230126.csv')
df2 = pd.read_csv('data/equiwidth_20230209.csv')
df = pd.concat([df0, df1, df2], axis=0)
df = df.fillna('TPC-DS')

## generate errors

In [9]:
def generate_errors(df):
    df = df.copy()  # Convenience
    
    df['absolute_error'] = np.absolute(df['estimated_output'] - df['real_output'])
    df['relative_error'] = df['absolute_error'] / df['real_output']
    df['x'] = df['estimated_output'] / df['real_output']
    df['1/x'] = 1 / df['x']
    df['q_error'] = df[['x','1/x']].max(axis=1)
    df['real_output_lower_bound'] = df['real_output']
    df['real_output_lower_bound'] = df['real_output_lower_bound'].clip(1)
    df['estimated_output_lower_bound'] = df['estimated_output']
    df['estimated_output_lower_bound'] = df['estimated_output_lower_bound'].clip(1)
    df['pseudo_x'] = df['estimated_output_lower_bound'] / df['real_output_lower_bound']
    df['pseudo_1/x'] = 1 / df['pseudo_x']
    df['pseudo_q_error'] = df[['pseudo_x','pseudo_1/x']].max(axis=1)
    df['mean_squared_error'] = df['absolute_error'] ** 2
    df.drop(['x', '1/x', 'real_output_lower_bound', 'estimated_output_lower_bound', 'pseudo_x', 'pseudo_1/x'], axis=1, inplace = True)

    return df

## result

### with_NULLs

In [11]:
df_with_errors = generate_errors(df)
df_with_errors

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

root_mean_squared_error  \
benchmark histogram                                              
JoinOrder EqualDistinctCountHistogram            443698.983120   
          EquiHeightHistogram                    492314.800920   
          EquiWidthHistogram                     496165.013349   
          MaxDiffFrequencyHistogram              498339.361284   
TPC-DS    EqualDistinctCountHistogram            162708.557941   
          EquiHeightHistogram                    159938.496916   
          EquiWidthHistogram                     166009.014377   
          MaxDiffFrequencyHistogram              162554.346247   
TPC-H     EqualDistinctCountHistogram              9039.773392   
          EquiHeightHistogram                      8745.382395   
          EquiWidthHistogram                       9033.915232   
          MaxDiffFrequencyHistogram                8825.051417   

                                       mean_absolute_error  \
benchmark histogram                                          
JoinOrder EqualDistinctCountHistogram        116215.915151   
          EquiHeightHistogram                118012.486780   
          EquiWidthHistogram                 118046.000538   
          MaxDiffFrequencyHistogram          121141.132950   
TPC-DS    EqualDistinctCountHistogram         61653.716376   
          EquiHeightHistogram                 60536.332173   
          EquiWidthHistogram                  64070.251770   
          MaxDiffFrequencyHistogram           61627.127810   
TPC-H     EqualDistinctCountHistogram          2468.485335   
          EquiHeightHistogram                  2185.476379   
          EquiWidthHistogram                   2471.827559   
          MaxDiffFrequencyHistogram            2357.633668   

                                       mean_relative_error  mean_q_error  \
benchmark histogram                                                        
JoinOrder EqualDistinctCountHistogram                  NaN           NaN   
          EquiHeightHistogram                          NaN           NaN   
          EquiWidthHistogram                           NaN           NaN   
          MaxDiffFrequencyHistogram                    NaN           NaN   
TPC-DS    EqualDistinctCountHistogram            22.099158           NaN   
          EquiHeightHistogram                    22.099502           NaN   
          EquiWidthHistogram                     21.618539           NaN   
          MaxDiffFrequencyHistogram              22.076917           NaN   
TPC-H     EqualDistinctCountHistogram                  NaN           NaN   
          EquiHeightHistogram                          NaN           NaN   
          EquiWidthHistogram                           NaN           NaN   
          MaxDiffFrequencyHistogram                    NaN           NaN   

                                       mean_pseudo_q_error  
benchmark histogram                                         
JoinOrder EqualDistinctCountHistogram         26461.034006  
          EquiHeightHistogram                 11692.064000  
          EquiWidthHistogram                  13092.666368  
          MaxDiffFrequencyHistogram           11149.412992  
TPC-DS    EqualDistinctCountHistogram          4657.808599  
          EquiHeightHistogram                  4899.630984  
          EquiWidthHistogram                   4175.518419  
          MaxDiffFrequencyHistogram            4618.700114  
TPC-H     EqualDistinctCountHistogram          1178.308454  
          EquiHeightHistogram                  1177.894999  
          EquiWidthHistogram                   1155.358496  
          MaxDiffFrequencyHistogram            1196.612079

### without_NULLs

In [12]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

Removed -248 (4.31%) rows because the 'real_output' cell was zero.
Removed -836 (14.52%) rows because the 'estimated_output' cell was zero.


root_mean_squared_error  \
benchmark histogram                                              
JoinOrder EqualDistinctCountHistogram            487677.353661   
          EquiHeightHistogram                    526581.363716   
          EquiWidthHistogram                     520019.381496   
          MaxDiffFrequencyHistogram              464675.441015   
TPC-DS    EqualDistinctCountHistogram            166481.168226   
          EquiHeightHistogram                    166386.585608   
          EquiWidthHistogram                     181287.694013   
          MaxDiffFrequencyHistogram              165742.867396   
TPC-H     EqualDistinctCountHistogram              5868.921448   
          EquiHeightHistogram                      5362.748174   
          EquiWidthHistogram                       6008.230648   
          MaxDiffFrequencyHistogram                5761.582905   

                                       mean_absolute_error  \
benchmark histogram                                          
JoinOrder EqualDistinctCountHistogram        135489.675520   
          EquiHeightHistogram                130868.741253   
          EquiWidthHistogram                 130165.147454   
          MaxDiffFrequencyHistogram          120347.838956   
TPC-DS    EqualDistinctCountHistogram         64074.571533   
          EquiHeightHistogram                 64734.529561   
          EquiWidthHistogram                  73882.754803   
          MaxDiffFrequencyHistogram           63898.491599   
TPC-H     EqualDistinctCountHistogram          1437.437634   
          EquiHeightHistogram                  1162.720831   
          EquiWidthHistogram                   1492.359613   
          MaxDiffFrequencyHistogram            1453.503820   

                                       mean_relative_error  mean_q_error  \
benchmark histogram                                                        
JoinOrder EqualDistinctCountHistogram           418.578825  2.687811e+06   
          EquiHeightHistogram                   611.899674  4.318380e+05   
          EquiWidthHistogram                    523.972262  4.768181e+05   
          MaxDiffFrequencyHistogram             572.079717  5.050956e+04   
TPC-DS    EqualDistinctCountHistogram            23.100213  2.819712e+08   
          EquiHeightHistogram                    23.851159  2.915432e+08   
          EquiWidthHistogram                     19.136653  3.798485e+04   
          MaxDiffFrequencyHistogram              22.916939  5.038439e+08   
TPC-H     EqualDistinctCountHistogram            40.477231  7.143097e+05   
          EquiHeightHistogram                    27.232247  1.146574e+06   
          EquiWidthHistogram                     21.564585  1.162059e+06   
          MaxDiffFrequencyHistogram              29.660478  1.394589e+06   

                                       mean_pseudo_q_error  
benchmark histogram                                         
JoinOrder EqualDistinctCountHistogram         28021.583955  
          EquiHeightHistogram                 10664.095566  
          EquiWidthHistogram                  12354.624118  
          MaxDiffFrequencyHistogram           10172.270333  
TPC-DS    EqualDistinctCountHistogram          4374.478350  
          EquiHeightHistogram                  4478.932424  
          EquiWidthHistogram                   2455.904064  
          MaxDiffFrequencyHistogram            4617.988887  
TPC-H     EqualDistinctCountHistogram            87.286465  
          EquiHeightHistogram                    74.506274  
          EquiWidthHistogram                     69.518191  
          MaxDiffFrequencyHistogram             102.711151

## compare_operators

In [6]:
grouped = df.groupby(["benchmark", "histogram", "query", "operator_type"]).size()

queries = df[["benchmark", "query"]].drop_duplicates()

# super inefficient, doesn't matter for now
op_counts = {}
for _, outer_benchmark, outer_query in queries.itertuples():
    for (_1, benchmark, histogram, query, operator_type, count) in grouped.reset_index().itertuples():
        if outer_benchmark == benchmark and outer_query == query:
            if (benchmark, query, operator_type) not in op_counts:
                op_counts[(benchmark, query, operator_type)] = count
                continue

            if op_counts[(benchmark, query, operator_type)] != count:
                print(f"WARNING: different value of {count} (previously {op_counts[(benchmark, query, operator_type)]}) for {benchmark}, query {query}, op {operator_type}")
